In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
main_dir = '/content/drive/My Drive/Facemask/New Masks Dataset'
train_dir = os.path.join(main_dir,'Train')
test_dir = os.path.join(main_dir,'Test')
valid_dir = os.path.join(main_dir,'Validation')

train_mask_dir = os.path.join(train_dir,'Mask')
train_nomask_dir = os.path.join(train_dir,'Non Mask')

In [ ]:
print(train_dir)

/content/drive/My Drive/Facemask/New Masks Dataset/Train


In [ ]:
train_mask_names = os.listdir(train_mask_dir)
print(train_mask_names[:10])

train_nomask_names = os.listdir(train_nomask_dir)
print(train_nomask_names[:10])


['0029.jpg', '0040.jpg', '0003.jpg', '0027.jpg', '0006.jpg', '0042.jpg', '0019.jpg', '0037.jpg', '0022.jpg', '0018.jpg']
['145.jpg', '115.jpg', '1.jpg', '118.jpg', '112.jpg', '102.jpg', '152.jpg', '153.jpg', '12.jpg', '127.jpg']


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range = 0.2,
                                   rotation_range = 40,
                                   horizontal_flip = True
                                   )
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                   target_size=(150,150),
                                   batch_size= 32,
                                   class_mode= 'binary'
                                   )
test_generator = test_datagen.flow_from_directory(test_dir,
                                   target_size=(150,150),
                                   batch_size= 32,
                                   class_mode= 'binary'
                                   )
valid_generator = validation_datagen.flow_from_directory(valid_dir,
                                   target_size=(150,150),
                                   batch_size= 32,
                                   class_mode= 'binary'
                                   )

Found 600 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Found 306 images belonging to 2 classes.


In [ ]:
train_generator.class_indices

{'Mask': 0, 'Non Mask': 1}

In [ ]:
train_generator.image_shape

(150, 150, 3)

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3),padding='SAME',activation='relu',input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))


model.add(Conv2D(64,(3,3),padding='SAME',activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 150, 150, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 75, 75, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 64)       0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 37, 37, 64)        0

In [ ]:
model.compile(Adam(lr=0.001),loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(train_generator,
                    epochs = 30,
                    validation_data = valid_generator)

Epoch 1/30
19/19 [==============================] - 214s 11s/step - loss: 3.8871 - accuracy: 0.5250 - val_loss: 0.6937 - val_accuracy: 0.4935
Epoch 2/30
19/19 [==============================] - 38s 2s/step - loss: 0.6006 - accuracy: 0.7283 - val_loss: 0.4518 - val_accuracy: 0.8366
Epoch 3/30
19/19 [==============================] - 38s 2s/step - loss: 0.3660 - accuracy: 0.8567 - val_loss: 0.3576 - val_accuracy: 0.8824
Epoch 4/30
19/19 [==============================] - 38s 2s/step - loss: 0.2962 - accuracy: 0.8950 - val_loss: 0.3420 - val_accuracy: 0.9085
Epoch 5/30
19/19 [==============================] - 38s 2s/step - loss: 0.2767 - accuracy: 0.9117 - val_loss: 0.3543 - val_accuracy: 0.8954
Epoch 6/30
19/19 [==============================] - 38s 2s/step - loss: 0.2620 - accuracy: 0.9033 - val_loss: 0.3382 - val_accuracy: 0.9020
Epoch 7/30
19/19 [==============================] - 38s 2s/step - loss: 0.2521 - accuracy: 0.9033 - val_loss: 0.3172 - val_accuracy: 0.8856
Epoch 8/30
19/19 [

In [ ]:
test_loss , test_acc = model.evaluate(test_generator)
print('test acc :{} test loss:{}'.format(test_acc,test_loss))

4/4 [==============================] - 20s 6s/step - loss: 0.1936 - accuracy: 0.9200
test acc :0.9200000166893005 test loss:0.19362185895442963


In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import cv2_imshow

uploaded=files.upload()
#print(uploaded)
for f in uploaded.keys():
  img_path='/content/'+f
  img = image.load_img(img_path , target_size=(150,150))
  images = image.img_to_array(img)
  images=np.expand_dims(images,axis=0)
  prediction = model.predict(images)
  if prediction==0:
    print(f ,'Mask is there')
  else:
    print(f ,'No mask is present')



Saving real_01072.jpg to real_01072.jpg
real_01072.jpg No mask is present


In [ ]:
model.save('newsaved_model.h5')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
